In [1]:
import { load } from "dotenv";
const env = await load();

const process = {
    env
}

In [2]:
import { ChatAlibabaTongyi } from "@langchain/community/chat_models/alibaba_tongyi";
import { BufferMemory } from "langchain/memory";
import { ChatPromptTemplate } from "@langchain/core/prompts";

const chatModel = new ChatAlibabaTongyi({
    verbose:true
});
const memory = new BufferMemory();

const TEMPLATE = `
你是一个乐于助人的 ai 助手。尽你所能回答所有问题。

这是跟人类沟通的聊天历史:
{history}

据此回答人类的问题:
{input}
`
const prompt = ChatPromptTemplate.fromTemplate(TEMPLATE);

In [3]:
import { RunnableSequence, RunnablePassthrough } from "@langchain/core/runnables";
import { StringOutputParser } from "@langchain/core/output_parsers";

let tempInput = ""

const chain =  RunnableSequence.from([
    {
        input: new RunnablePassthrough(),
        memoryObject: async (input) => {
            const history = await memory.loadMemoryVariables({
                input
            })
            tempInput = input
            return history
        }
    },
    RunnablePassthrough.assign({
        history: (input) => input.memoryObject.history
    }),
    prompt,
    chatModel,
    new StringOutputParser(),
    new RunnablePassthrough({
        func: async (output) => {
            await memory.saveContext({
                input: tempInput
            }, {
                output
            })
        }
    }),
]);

In [4]:
const res1 = await chain.invoke("你好, 我叫小明");
console.log(res1);

[llm/start] [1:llm:ChatAlibabaTongyi] Entering LLM run with input: {
  "messages": [
    [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain_core",
          "messages",
          "HumanMessage"
        ],
        "kwargs": {
          "content": "\n你是一个乐于助人的 ai 助手。尽你所能回答所有问题。\n\n这是跟人类沟通的聊天历史:\n\n\n据此回答人类的问题:\n你好, 我叫小明\n",
          "additional_kwargs": {},
          "response_metadata": {}
        }
      }
    ]
  ]
}
[llm/end] [1:llm:ChatAlibabaTongyi] [1.38s] Exiting LLM run with output: {
  "generations": [
    [
      {
        "text": "你好，小明！很高兴为你提供帮助。有什么可以帮到你的吗？",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain_core",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "你好，小明！很高兴为你提供帮助。有什么可以帮到你的吗？",
            "tool_calls": [],
            "invalid_tool_calls": [],
            "additional_kwargs": {},
            "re

In [5]:
await chain.invoke("我叫什么？");

[llm/start] [1:llm:ChatAlibabaTongyi] Entering LLM run with input: {
  "messages": [
    [
      {
        "lc": 1,
        "type": "constructor",
        "id": [
          "langchain_core",
          "messages",
          "HumanMessage"
        ],
        "kwargs": {
          "content": "\n你是一个乐于助人的 ai 助手。尽你所能回答所有问题。\n\n这是跟人类沟通的聊天历史:\nHuman: 你好, 我叫小明\nAI: 你好，小明！很高兴为你提供帮助。有什么可以帮到你的吗？\n\n据此回答人类的问题:\n我叫什么？\n",
          "additional_kwargs": {},
          "response_metadata": {}
        }
      }
    ]
  ]
}
[llm/end] [1:llm:ChatAlibabaTongyi] s] Exiting LLM run with output: {
  "generations": [
    [
      {
        "text": "你叫小明。",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain_core",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "你叫小明。",
            "tool_calls": [],
            "invalid_tool_calls": [],
            "additional_kwargs": {},
            "respon

"你叫小明。"

In [6]:
import { JSONChatHistory } from "./JSONChatHistory/index.ts"
import { AIMessage, HumanMessage } from "@langchain/core/messages";

const history = new JSONChatHistory({
    dir: "chat_data",
    sessionId: "test"
})


await history.addMessages([
  new HumanMessage("Hi, 我叫小明"),
  new AIMessage("你好"),
]);

const messages = await history.getMessages();
console.log(messages)


[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "Hi, 我叫小明",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "Hi, 我叫小明",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: { content: "你好", additional_kwargs: {}, response_metadata: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "你好",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  }
]


In [7]:
import { ChatAlibabaTongyi } from "@langchain/community/chat_models/alibaba_tongyi";
import { BufferMemory } from "langchain/memory";
import { ConversationChain } from "langchain/chains";


const chatModel = new ChatAlibabaTongyi();
const memory = new BufferMemory({
    chatHistory: history
});
const chain = new ConversationChain({ llm: chatModel, memory: memory });
const res1 = await chain.call({ input: "我叫什么？" });


In [8]:
console.log(res1);

{ response: "你叫小明。" }


In [9]:
const messages = await history.getMessages();

In [10]:
console.log(messages);

[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "Hi, 我叫小明",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "Hi, 我叫小明",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: { content: "你好", additional_kwargs: {}, response_metadata: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "你好",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: { content: "我叫什么？", additional_kwargs: {}, response_metadata: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "我叫什么？",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: { content: "你叫小明。", additional_kwargs: {}, response_metadata: {} },
    lc_namespac